###Тема “POS-tagger и NER”

####Задание 1. 
Написать теггер на данных с русским языком
- проверить UnigramTagger, BigramTagger, TrigramTagger и их комбмнации
- написать свой теггер как на занятии, попробовать разные векторайзеры, добавить знание не только букв но и слов
- сравнить все реализованные методы сделать выводы
 
####Задание 2. 
- Проверить насколько хорошо работает NER
данные брать из http://www.labinform.ru/pub/named_entities/
- проверить NER из nltk/spacy/deeppavlov
- написать свой нер попробовать разные подходы
  - передаём в сетку токен и его соседей
  - передаём в сетку только токен
  - свой вариант
- сравнить ваши реализованные подходы на качество (вывести precision/recall/f1_score)


In [1]:
#для проверки теггеров воспользуемся размеченными вручную данными.
#такой результат будет более релевантен.

!wget -O ru_syntagrus-ud-train.conllu https://raw.githubusercontent.com/UniversalDependencies/UD_Russian-SynTagRus/master/ru_syntagrus-ud-train-a.conllu
!wget -O ru_syntagrus-ud-dev.conllu https://raw.githubusercontent.com/UniversalDependencies/UD_Russian-SynTagRus/master/ru_syntagrus-ud-dev.conllu

--2022-07-15 06:42:42--  https://raw.githubusercontent.com/UniversalDependencies/UD_Russian-SynTagRus/master/ru_syntagrus-ud-train-a.conllu
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 40736565 (39M) [text/plain]
Saving to: ‘ru_syntagrus-ud-train.conllu’

ru_syntagrus-ud-tra 100%[===================>]  38.85M   255MB/s    in 0.2s    

2022-07-15 06:42:45 (255 MB/s) - ‘ru_syntagrus-ud-train.conllu’ saved [40736565/40736565]

--2022-07-15 06:42:45--  https://raw.githubusercontent.com/UniversalDependencies/UD_Russian-SynTagRus/master/ru_syntagrus-ud-dev.conllu
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.111.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108

In [2]:
!pip install pyconll

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
import pandas as pd
import numpy as np
import re

from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.feature_extraction.text import CountVectorizer, HashingVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

import pyconll
import nltk
from nltk.tag import DefaultTagger, UnigramTagger, BigramTagger, TrigramTagger

import warnings
warnings.filterwarnings("ignore")

In [4]:
data_train = pyconll.load_from_file('ru_syntagrus-ud-train.conllu')
data_test = pyconll.load_from_file('ru_syntagrus-ud-dev.conllu')

In [5]:
fdata_train = []
for sent in data_train[:]:
    fdata_train.append([(token.form, token.upos) for token in sent])
    
fdata_test = []
for sent in data_test[:]:
    fdata_test.append([(token.form, token.upos) for token in sent])
    
fdata_sent_test = []
for sent in data_test[:]:
    fdata_sent_test.append([token.form for token in sent])

In [6]:
len(fdata_train), len(fdata_test), len(fdata_sent_test)

(24516, 8906, 8906)

In [7]:
fdata_train[:2]

[[('Анкета', 'NOUN'), ('.', 'PUNCT')],
 [('Начальник', 'NOUN'),
  ('областного', 'ADJ'),
  ('управления', 'NOUN'),
  ('связи', 'NOUN'),
  ('Семен', 'PROPN'),
  ('Еремеевич', 'PROPN'),
  ('был', 'AUX'),
  ('человек', 'NOUN'),
  ('простой', 'ADJ'),
  (',', 'PUNCT'),
  ('приходил', 'VERB'),
  ('на', 'ADP'),
  ('работу', 'NOUN'),
  ('всегда', 'ADV'),
  ('вовремя', 'ADV'),
  (',', 'PUNCT'),
  ('здоровался', 'VERB'),
  ('с', 'ADP'),
  ('секретаршей', 'NOUN'),
  ('за', 'ADP'),
  ('руку', 'NOUN'),
  ('и', 'CCONJ'),
  ('иногда', 'ADV'),
  ('даже', 'PART'),
  ('писал', 'VERB'),
  ('в', 'ADP'),
  ('стенгазету', 'NOUN'),
  ('заметки', 'NOUN'),
  ('под', 'ADP'),
  ('псевдонимом', 'NOUN'),
  ('"', 'PUNCT'),
  ('Муха', 'NOUN'),
  ('"', 'PUNCT'),
  ('.', 'PUNCT')]]

In [8]:

unigram_tagger = UnigramTagger(fdata_train)
unigram_acc = unigram_tagger.evaluate(fdata_test)

bigram_tagger = BigramTagger(fdata_train)
bigram_acc = bigram_tagger.evaluate(fdata_test)

trigram_tagger = TrigramTagger(fdata_train)
trigram_acc = trigram_tagger.evaluate(fdata_test)

bigram_tagger = BigramTagger(fdata_train, backoff=unigram_tagger)
bigram_unigram_acc = bigram_tagger.evaluate(fdata_test)

trigram_tagger = TrigramTagger(fdata_train, backoff=bigram_tagger)
trigram_bigram_unigram_acc = trigram_tagger.evaluate(fdata_test)

print(f'Accuracy:\nUnigram Tagger: {round(unigram_acc, 3)},\nBigram Tagger: {round(bigram_acc, 5)},\n'
      f'Trigram Tagger: {round(trigram_acc, 3)},\nBigram and Unigram Tagger: {round(bigram_unigram_acc, 5)},\n'
      f'Trigram, Bigram and Unigram Tagger: {round(trigram_bigram_unigram_acc, 5)},\n')

Accuracy:
Unigram Tagger: 0.824,
Bigram Tagger: 0.60939,
Trigram Tagger: 0.178,
Bigram and Unigram Tagger: 0.82928,
Trigram, Bigram and Unigram Tagger: 0.82914,



Комбинация теггеров в данном случае дала некоторый прирост качества

####Попробуем написать свой теггер

In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer, HashingVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score, classification_report
from sklearn.preprocessing import LabelEncoder

In [10]:
# Переведём тренировочный датасет в списки слов и списки POS-разметки
train_tok = []
train_label = []
for sent in fdata_train[:]:
    for tok in sent:
        train_tok.append(tok[0])
        train_label.append('NO_TAG' if tok[1] is None else tok[1])
        
test_tok = []
test_label = []
for sent in fdata_test[:]:
    for tok in sent:
        test_tok.append(' ' if tok[0] is None else tok[0])
        test_label.append('NO_TAG' if tok[1] is None else tok[1])

In [11]:
train_tok[:5], train_label[:5]

(['Анкета', '.', 'Начальник', 'областного', 'управления'],
 ['NOUN', 'PUNCT', 'NOUN', 'ADJ', 'NOUN'])

In [12]:
le = LabelEncoder()
train_enc_labels = le.fit_transform(train_label)
train_enc_labels

array([ 7, 13,  7, ...,  1, 11, 13])

In [13]:
test_enc_labels = le.transform(test_label)
test_enc_labels

array([ 7, 13,  1, ...,  0,  7, 13])

In [14]:
le.classes_

array(['ADJ', 'ADP', 'ADV', 'AUX', 'CCONJ', 'DET', 'INTJ', 'NOUN',
       'NO_TAG', 'NUM', 'PART', 'PRON', 'PROPN', 'PUNCT', 'SCONJ', 'SYM',
       'VERB', 'X'], dtype='<U6')

In [15]:
%time

vectorizers = [CountVectorizer(ngram_range=(1, 3), analyzer='char'), 
               TfidfVectorizer(ngram_range=(1, 3), analyzer='char'), 
               HashingVectorizer(ngram_range=(1, 3), analyzer='char', n_features=1000)] 
vectorizers_word = [CountVectorizer(ngram_range=(1, 3), analyzer='word'), 
               TfidfVectorizer(ngram_range=(1, 3), analyzer='word'), 
               HashingVectorizer(ngram_range=(1, 3), analyzer='word', n_features=1000)] 
n_features = [2000, 3000, 5000]
vectorizers_hash = [HashingVectorizer(ngram_range=(1, 3), analyzer='char', n_features=feat) for feat in n_features]
vectorizers_hash_word = [HashingVectorizer(ngram_range=(1, 3), analyzer='word', n_features=feat) for feat in n_features]
f1_scores = []
accuracy_scores = []

for vectorizer in vectorizers + vectorizers_word + vectorizers_hash + vectorizers_hash_word:
    X_train = vectorizer.fit_transform(train_tok)
    X_test = vectorizer.transform(test_tok)
    
    lr = LogisticRegression(random_state=0, max_iter=100)
    lr.fit(X_train, train_enc_labels)
    pred = lr.predict(X_test)
    f1 = f1_score(test_enc_labels, pred, average='weighted')
    f1_scores.append(f1)
    acc = accuracy_score(test_enc_labels, pred)
    accuracy_scores.append(acc)
    
    print(vectorizer)
    print(classification_report(test_enc_labels, pred, target_names=le.classes_))

CPU times: user 4 µs, sys: 1 µs, total: 5 µs
Wall time: 8.11 µs
CountVectorizer(analyzer='char', ngram_range=(1, 3))
              precision    recall  f1-score   support

         ADJ       0.91      0.91      0.91     15103
         ADP       0.98      1.00      0.99     13717
         ADV       0.90      0.90      0.90      7783
         AUX       0.81      0.97      0.88      1390
       CCONJ       0.88      0.98      0.93      5672
         DET       0.89      0.73      0.80      4265
        INTJ       0.35      0.29      0.32        24
        NOUN       0.92      0.95      0.93     36238
      NO_TAG       1.00      0.77      0.87       265
         NUM       0.84      0.90      0.87      1734
        PART       0.95      0.76      0.85      5125
        PRON       0.83      0.90      0.86      7444
       PROPN       0.75      0.58      0.66      5473
       PUNCT       1.00      1.00      1.00     29186
       SCONJ       0.80      0.91      0.85      2865
         SYM      

In [16]:
result_model = pd.DataFrame({'Vectorizer': vectorizers + vectorizers_word + vectorizers_hash + vectorizers_hash_word,
                            'f1_score': f1_scores})
result_model.sort_values('f1_score', ascending=False)

,Vectorizer,f1_score
0,"CountVectorizer(analyzer='char', ngram_range=(...",0.924048
1,"TfidfVectorizer(analyzer='char', ngram_range=(...",0.917500
8,"HashingVectorizer(analyzer='char', n_features=...",0.897982
7,"HashingVectorizer(analyzer='char', n_features=...",0.893299
6,"HashingVectorizer(analyzer='char', n_features=...",0.890910
2,"HashingVectorizer(analyzer='char', n_features=...",0.876592
4,"TfidfVectorizer(ngram_range=(1, 3))",0.654762
3,"CountVectorizer(ngram_range=(1, 3))",0.637163
11,"HashingVectorizer(n_features=5000, ngram_range...",0.585894
10,"HashingVectorizer(n_features=3000, ngram_range...",0.568290


In [17]:
result_model_acc = pd.DataFrame({'Vectorizer': vectorizers + vectorizers_word + vectorizers_hash + vectorizers_hash_word,
                            'Accuracy': accuracy_scores})
result_model_acc.sort_values('Accuracy', ascending=False)


,Vectorizer,Accuracy
0,"CountVectorizer(analyzer='char', ngram_range=(...",0.925802
1,"TfidfVectorizer(analyzer='char', ngram_range=(...",0.919917
8,"HashingVectorizer(analyzer='char', n_features=...",0.901543
7,"HashingVectorizer(analyzer='char', n_features=...",0.896771
6,"HashingVectorizer(analyzer='char', n_features=...",0.894511
2,"HashingVectorizer(analyzer='char', n_features=...",0.879510
4,"TfidfVectorizer(ngram_range=(1, 3))",0.644176
3,"CountVectorizer(ngram_range=(1, 3))",0.628068
11,"HashingVectorizer(n_features=5000, ngram_range...",0.604213
10,"HashingVectorizer(n_features=3000, ngram_range...",0.592786


###Задание 2

In [18]:
!pip install corus

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 83 kB 1.8 MB/s 


In [19]:
import corus
from corus import load_ne5
import nltk

In [20]:
nltk.download('maxent_ne_chunker')
nltk.download('words')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping chunkers/maxent_ne_chunker.zip.
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [21]:
!wget http://www.labinform.ru/pub/named_entities/collection5.zip

--2022-07-15 07:01:54--  http://www.labinform.ru/pub/named_entities/collection5.zip
Resolving www.labinform.ru (www.labinform.ru)... 95.181.230.181
Connecting to www.labinform.ru (www.labinform.ru)|95.181.230.181|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1899530 (1.8M) [application/zip]
Saving to: ‘collection5.zip’

collection5.zip     100%[===================>]   1.81M  1.50MB/s    in 1.2s    

2022-07-15 07:01:56 (1.50 MB/s) - ‘collection5.zip’ saved [1899530/1899530]



In [ ]:
!unzip collection5.zip

In [23]:
records = load_ne5('Collection5/')
document = next(records).text
document

'Президент России уволил 6 генералов МВД\r\n \r\nПрезидент России Дмитрий Медведев продолжает перестановки в руководстве МВД. Сегодня он уволил 6 генералов милиции.\r\n\r\nВ частности, указом президента отправлен в отставку начальник Центра обеспечения оперативно-служебной деятельности по противодействию экстремизму МВД генерал-майор милиции Владимир Булатов. Не взяли на работу в полицию и ряд региональных руководителей ведомства. Уволены начальник Управления на транспорте МВД по Северо-Кавказскому федеральному округу генерал-майор милиции Игорь Жуков, начальник Управления внутренних дел по Тверской области генерал-майор милиции Александр Куликов, начальник Управления внутренних дел по Тамбовской области генерал-майор милиции Владимир Фурсов.\r\n\r\nТакже президент отправил в отставку начальников 2 милицейских институтов: начальника Сибирского юридического института МВД РФ генерал-майора милиции Николая Михайлова и начальника Тюменского юридического института МВД генерал-майора милиции

In [24]:
{(' '.join(c[0] for c in chunk), chunk.label() ) for chunk in nltk.ne_chunk(nltk.pos_tag(nltk.word_tokenize(document))) if hasattr(chunk, 'label') }


{('Николая Михайлова', 'PERSON'),
 ('Президент', 'PERSON'),
 ('России', 'ORGANIZATION'),
 ('России Дмитрий Медведев', 'PERSON'),
 ('Сибирского', 'ORGANIZATION'),
 ('Управления', 'ORGANIZATION'),
 ('Центра', 'PERSON')}

В целом распознано верно, но есть погрешности

####Spacy

In [27]:
!pip install spacy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [28]:
!python -m spacy download ru_core_news_sm

2022-07-15 07:08:25.389093: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 15.3 MB 7.6 MB/s 
     |████████████████████████████████| 55 kB 3.4 MB/s 
     |████████████████████████████████| 8.2 MB 11.5 MB/s 
✔ Download and installation successful
You can now load the package via spacy.load('ru_core_news_sm')


In [30]:
import spacy
from spacy import displacy
import ru_core_news_sm

In [32]:
nlp = ru_core_news_sm.load()
ny_bb = document
article = nlp(ny_bb)
displacy.render(article, jupyter=True, style='ent')

In [33]:
#список токенов, частей речи и сущностей
for token in article:
    print(token.text, token.pos_, token.dep_)

Президент NOUN nsubj
России PROPN nmod
уволил VERB ROOT
6 NUM nummod
генералов NOUN obj
МВД PROPN nmod

 
 SPACE dep
Президент NOUN nsubj
России PROPN nmod
Дмитрий PROPN appos
Медведев PROPN flat:name
продолжает VERB conj
перестановки NOUN obj
в ADP case
руководстве NOUN nmod
МВД PROPN nmod
. PUNCT punct
Сегодня ADV advmod
он PRON nsubj
уволил VERB ROOT
6 NUM nummod
генералов NOUN obj
милиции NOUN nmod
. PUNCT punct


 SPACE dep
В ADP discourse
частности NOUN fixed
, PUNCT punct
указом NOUN obl
президента NOUN nmod
отправлен VERB ROOT
в ADP case
отставку NOUN obl
начальник NOUN nsubj:pass
Центра PROPN nmod
обеспечения NOUN nmod
оперативно ADJ amod
- ADJ amod
служебной ADJ amod
деятельности NOUN nmod
по ADP case
противодействию NOUN nmod
экстремизму NOUN iobj
МВД PROPN nmod
генерал NOUN appos
- NOUN appos
майор NOUN appos
милиции NOUN nmod
Владимир PROPN appos
Булатов PROPN flat:name
. PUNCT punct
Не PART advmod
взяли VERB ROOT
на ADP case
работу NOUN obl
в ADP case
полицию NOUN obl
и C

#### DeepPavlov

In [34]:
import tensorflow as tf
print(tf.version.VERSION)

2.8.2


In [ ]:
!pip install numpy scipy librosa unidecode inflect librosa transformers
!pip install deeppavlov

In [ ]:
!python -m deeppavlov install squad_bert
!python -m deeppavlov install ner_ontonotes

In [38]:
import deeppavlov
from deeppavlov import configs, build_model

In [ ]:
deeppavlov_ner = build_model(configs.ner.ner_bert_ent_and_type_rus, download=True)
deeppavlov_ner(document)

#### С deeppavlov не удалось разобраться -- много ошибок с зависимостями и видимо надо отдельно погружаться в тему

### Свой NER

In [40]:
!pip install razdel

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [41]:
from razdel import tokenize

In [42]:
words_docs = []
for ix, rec in enumerate(records):
    words = []
    for token in tokenize(rec.text):
        
        type_ent = 'OUT'
        for ent in rec.spans:
            if (token.start >= ent.start) and (token.stop <= ent.stop):
                type_ent = ent.type
                break
        words.append([token.text, type_ent])
    words_docs.extend(words)

In [43]:
df_words = pd.DataFrame(words_docs, columns=['word', 'tag'])

In [44]:
df_words['tag'].value_counts()

OUT         219139
PER          21186
ORG          13623
LOC           4561
GEOPOLIT      4353
MEDIA         2482
Name: tag, dtype: int64

In [45]:
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D, GlobalMaxPooling1D, Conv1D, GRU, LSTM, Dropout, Input, Bidirectional,Reshape
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
from sklearn import model_selection, preprocessing, linear_model

In [46]:
train_x, valid_x, train_y, valid_y = model_selection.train_test_split(df_words['word'], df_words['tag'])

# labelEncode
encoder = preprocessing.LabelEncoder()
train_y = encoder.fit_transform(train_y)
valid_y = encoder.fit_transform(valid_y)

In [47]:
train_x.apply(len).max(axis=0)

32

In [48]:
train_data = tf.data.Dataset.from_tensor_slices((train_x, train_y))
valid_data = tf.data.Dataset.from_tensor_slices((valid_x, valid_y))

train_data = train_data.batch(2048)
valid_data = valid_data.batch(2048)

In [49]:
AUTOTUNE = tf.data.AUTOTUNE

train_data = train_data.cache().prefetch(buffer_size=AUTOTUNE)
valid_data = valid_data.cache().prefetch(buffer_size=AUTOTUNE)

In [50]:
def custom_standardization(input_data):
    return input_data

vocab_size = 30000
seq_len = 10


vectorize_layer = TextVectorization(
    standardize=custom_standardization,
    max_tokens=vocab_size,
    output_mode='int',
    output_sequence_length=seq_len
    )

text_data = train_data.map(lambda x, y: x)
vectorize_layer.adapt(text_data)

In [51]:
t=np.unique(encoder.inverse_transform(valid_y),return_counts=True)[1]
t=t/t.sum()
t  

array([0.01685359, 0.01748673, 0.00838157, 0.05123915, 0.82578087,
       0.08025808])

In [53]:
model = tf.keras.Sequential([
  vectorize_layer,
  tf.keras.layers.Embedding(len(vectorize_layer.get_vocabulary()), 64, mask_zero=True),
  tf.keras.layers.GlobalAveragePooling1D(),
  tf.keras.layers.Dense(300, activation='relu'),
  tf.keras.layers.Dropout(0.5),
  tf.keras.layers.Dense(50, activation='relu'),
  tf.keras.layers.Dense(6, activation='softmax')
  ])

In [54]:
model.compile(optimizer='adam',         
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])
model.fit(train_data, validation_data=valid_data, epochs=5)

Epoch 1/5
98/98 [==============================] - 8s 69ms/step - loss: 0.7737 - accuracy: 0.8235 - val_loss: 0.3918 - val_accuracy: 0.8683
Epoch 2/5
98/98 [==============================] - 5s 48ms/step - loss: 0.2621 - accuracy: 0.9122 - val_loss: 0.2547 - val_accuracy: 0.9324
Epoch 3/5
98/98 [==============================] - 5s 49ms/step - loss: 0.1484 - accuracy: 0.9559 - val_loss: 0.2377 - val_accuracy: 0.9405
Epoch 4/5
98/98 [==============================] - 5s 54ms/step - loss: 0.1196 - accuracy: 0.9633 - val_loss: 0.2335 - val_accuracy: 0.9414
Epoch 5/5
98/98 [==============================] - 5s 49ms/step - loss: 0.1116 - accuracy: 0.9644 - val_loss: 0.2324 - val_accuracy: 0.9412


In [55]:
pred=model.predict(valid_data)

In [56]:
list(zip(encoder.classes_,pred.mean(axis=0)/t))

[('GEOPOLIT', 0.9981119692965357),
 ('LOC', 0.9751331507131971),
 ('MEDIA', 1.522513045949473),
 ('ORG', 1.3357483595741184),
 ('OUT', 0.990347254078025),
 ('PER', 0.83858281333674)]

In [57]:
model = tf.keras.Sequential([
    vectorize_layer,
    tf.keras.layers.Embedding(len(vectorize_layer.get_vocabulary()), 64, mask_zero=True),
    tf.keras.layers.Bidirectional(tf.keras.layers.GRU(64)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(6,activation='softmax')
])

In [58]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])

In [59]:
model.fit(train_data, validation_data=valid_data, epochs=5)

Epoch 1/5
98/98 [==============================] - 46s 354ms/step - loss: 0.8406 - accuracy: 0.8156 - val_loss: 0.4179 - val_accuracy: 0.8424
Epoch 2/5
98/98 [==============================] - 32s 323ms/step - loss: 0.3210 - accuracy: 0.8989 - val_loss: 0.2730 - val_accuracy: 0.9178
Epoch 3/5
98/98 [==============================] - 32s 324ms/step - loss: 0.2027 - accuracy: 0.9399 - val_loss: 0.2258 - val_accuracy: 0.9355
Epoch 4/5
98/98 [==============================] - 32s 332ms/step - loss: 0.1549 - accuracy: 0.9555 - val_loss: 0.2103 - val_accuracy: 0.9398
Epoch 5/5
98/98 [==============================] - 31s 321ms/step - loss: 0.1340 - accuracy: 0.9617 - val_loss: 0.2088 - val_accuracy: 0.9408
